# Homework 2 -Data from the Web

Useful imports first

In [4]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import html

We want to get first all the parameters that are possible, i.e. we will fill two dictionnaries, one with the year as index, and the "value" of the link associated to it (e.g. {'2016-2017': '355925344'}), and the same with the bachelor year and the "value" of the link associated to it.

We do that in order to be able to extract all students from all possible years (2007-2016) and from all stages of the bachelor automatically later.

In [5]:
# The URL of the "start" of the IS-Academia page listing the students.
home_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_x_GPS=-1&ww_i_reportModel=133685247"

# The general form of the URL, with the fields to be replaced later (e.g. [UNITE_ACADEMIQUE])
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=[UNITE_ACADEMIQUE]&ww_x_PERIODE_ACAD=[PERIODE_ACADEMIQUE]&ww_x_PERIODE_PEDAGO=[PERIODE_PEDAGOGIQUE]&ww_x_HIVERETE=null"

# Get the raw content from the page
with urllib.request.urlopen(home_url) as url:
    s = url.read()

soup = BeautifulSoup(s, 'html.parser')    

# Parse the content
url_section = 0
url_years = {}
url_bachelor = {}

#For loop on all the possible fields with <option> to get the value associated with each year, or each "Bachelor" semester
for link in soup.find_all('option'):
    if link.text == 'Informatique':
        url_section = link.get('value')
    if "20" in link.text:
        #print(link.text)
        url_years[link.text] = link.get('value')
    if "Bachelor" in link.text:
        #print(link.text)
        url_bachelor[link.text] = link.get('value')
        #get('value'))

        
print(url_years)
print(url_bachelor)
#frame= frame.drop([2,3,4,5,6,8,9,11],axis=1)
#frame.head(10)

{'2008-2009': '978187', '2010-2011': '39486325', '2009-2010': '978195', '2011-2012': '123455150', '2013-2014': '213637754', '2014-2015': '213637922', '2015-2016': '213638028', '2007-2008': '978181', '2012-2013': '123456101', '2016-2017': '355925344'}
{'Bachelor semestre 4': '942163', 'Bachelor semestre 6': '942175', 'Bachelor semestre 2': '249114', 'Bachelor semestre 1': '249108', 'Bachelor semestre 5b': '2226768', 'Bachelor semestre 3': '942155', 'Bachelor semestre 5': '942120', 'Bachelor semestre 6b': '2226785'}


Function that replaces the general fields in the base_url with the fields for a given year and level of study

In [6]:
def getFullUrl(PeriodeAcad, PeriodePedag):
    url = base_url
    url = url.replace('[UNITE_ACADEMIQUE]', str(url_section))
    url = url.replace('[PERIODE_ACADEMIQUE]', str(url_years[PeriodeAcad]))
    url = url.replace('[PERIODE_PEDAGOGIQUE]', str(url_bachelor[PeriodePedag]))
    return url

print(getFullUrl('2007-2008', 'Bachelor semestre 1'))


http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=978181&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=null


In [83]:
#url = getFullUrl('2007-2008', 'Bachelor semestre 1')

#with urllib.request.urlopen(url) as url:
#    s = url.read()
#soup = BeautifulSoup(s, 'html.parser')    
#print(soup.prettify())
#student_frame = pd.read_html(soup.prettify(), header=1)

#frame = student_frame[0].drop(0,axis=0)
#frame = frame[['Civilité', 'Nom Prénom', 'No Sciper']]

## Load all student from that have started EPFL from 2007 in current_student data frame
## and all students that have finished from 2007 in finished_student data frame

current_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 'Starting year'])
finished_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 'Finishing year'])


for year in range(10):
    current_year = 2007+year
    year_string = str(current_year) + '-' + str(current_year+1)
    
    # Get the data from the appropriate url
    current_url_B1 = getFullUrl(year_string, 'Bachelor semestre 1')
    with urllib.request.urlopen(current_url_B1) as url:
        html_B1 = url.read()
    soup = BeautifulSoup(html_B1, 'html.parser')
    
    # Create the data frame only keeping relevant fields
    student_frame_B1 = pd.read_html(soup.prettify(), header=1)
    student_frame_B1 = student_frame_B1[0].drop(0,axis=0)
    student_frame_B1 = student_frame_B1[['Civilité', 'Nom Prénom', 'No Sciper']]
    student_frame_B1.index = student_frame_B1['No Sciper']
    student_frame_B1 = student_frame_B1.drop('No Sciper', axis=1)
    student_frame_B1['Starting year'] = current_year;
    # Only keep students that are in their first 1st year
    student_frame_B1 = student_frame_B1[[not (sciper in current_student.index) for sciper in student_frame_B1.index]]
    current_student = pd.concat([current_student, student_frame_B1])
    
    # Get the data from the appropriate url
    current_url_B6 = getFullUrl(year_string, 'Bachelor semestre 6')
    with urllib.request.urlopen(current_url_B6) as url:
        html_B6 = url.read()
    soup = BeautifulSoup(html_B6, 'html.parser')
    
    # Create the data frame only keeping relevant fields
    student_frame_B6 = pd.read_html(soup.prettify(), header=1)
    student_frame_B6 = student_frame_B6[0].drop(0,axis=0)
    student_frame_B6 = student_frame_B6[['Civilité', 'Nom Prénom', 'No Sciper']]
    student_frame_B6.index = student_frame_B6['No Sciper']
    student_frame_B6 = student_frame_B6.drop('No Sciper', axis=1)
    # We replace students 
    current_student[[(sciper in finished_student_sciper) for sciper in current_student.index]]
    finished_student = finished_student[[not (sciper in student_frame_B6.index) for sciper in  finished_student.index]]
    student_frame_B6['Finishing year'] = current_year;
    finished_student = pd.concat([finished_student, student_frame_B6])
        
#print(current_student.head(5))
#print(current_student.tail(5))
print(current_student.shape)
print(current_student.index.is_unique)
print(finished_student.index.is_unique)
print(finished_student.head(100))
#print(finished_student.tail(5))


(1317, 3)
True
True
           Civilité                         Nom Prénom  Finishing year
No Sciper                                                             
161634     Monsieur                   Boksanyi Michael            2007
170451     Monsieur                 Bolkensteyn Dinesh            2007
170219     Monsieur        Bornet Cyril Antoine Michel            2007
153762     Monsieur                       Conus Johann            2007
166548     Monsieur                     Delort Vincent            2007
166746     Monsieur                     Donzé Benjamin            2007
166748     Monsieur                 Dos Santos Patrick            2007
166751     Monsieur                       Droz Olivier            2007
170414     Monsieur                      Eberle Julien            2007
166889     Monsieur                        Genet Henry            2007
161091       Madame                   Grivet Ekaterina            2007
159998       Madame                        Jesse Julia   

In [78]:
finished_student_sciper = finished_student.index

# Only keep students that started after 2007.
finished_student_from_2007 = current_student.loc[finished_student_sciper]

print(current_student.shape)
print(finished_student.shape)
print(finished_student_sciper.shape)
print(finished_student_from_2007.shape)
print(finished_student.index.is_unique)

print(finished_student_sciper)

#print(finished_student.duplicated, 'Nom Prénom')
#print(finished_student)
#print(finished_student[[gender.endswith('Madame') for gender in finished_student.Civilité]])

print(current_student[[(sciper in finished_student_sciper) for sciper in current_student.index]])
#data[[name.endswith('bacteria') for name in data.phylum] and data.value > 1000]



(1317, 3)
(654, 3)
(654,)
(654, 3)
False
Int64Index([166701, 161279, 166344, 161634, 170451, 170219, 161814, 153762,
            166548, 166746,
            ...
            247855, 227311, 245782, 247233, 225452, 250362, 235935, 247680,
            246532, 247891],
           dtype='int64', length=654)
        Civilité                          Nom Prénom  Starting year
174905  Monsieur                     Aubelle Flavien         2007.0
179406  Monsieur                  Baeriswyl Jonathan         2007.0
179428  Monsieur                     Barroco Michael         2007.0
179449  Monsieur               Bindschaedler Vincent         2007.0
178553  Monsieur                  Bloch Marc-Olivier         2007.0
179426  Monsieur                          Bloch Remi         2007.0
178271  Monsieur              Boéchat Marc-Alexandre         2007.0
180731  Monsieur                Bricola Jean-Charles         2007.0
171619  Monsieur                Buchschacher Nicolas         2007.0
179157  Monsieur